In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from tqdm import tqdm

from src.data_utils import load_and_preprocess, split_data
from src.next_token_dataset import NextTokenDataset
from src.lstm_model import LSTMAutocomplete
from src.lstm_train import LSTMTrainer
from src.eval_lstm import evaluate_rouge
from src.eval_transformer_pipeline import evaluate_gpt2_rouge

import yaml

In [2]:
with open("configs/lstm.yaml", "r") as f:
    config = yaml.safe_load(f)

RANDOM_STATE = config["general"]["random_state"] #42

NUM_EPOCHS = config["training"]["num_epochs"] #3

SEQ_LEN = config["data"]["seq_len"] #20

Выполним загрузку и обработку текстовых данных, просмотрим первые строки:

In [3]:
df = load_and_preprocess("data/raw_dataset.txt")

df.head()

0    awww that s a bummer you shoulda got david car...
1    is upset that he can t update his facebook by ...
2    i dived many times for the ball managed to sav...
3       my whole body feels itchy and like its on fire
4    no it s not behaving at all i m mad why am i h...
Name: cleaned_text, dtype: object

Выполним разбиение на обучающую, тренировочную и валидационную выборки и выведем их размер:

In [4]:
train_texts, val_texts, test_texts = split_data(df, RANDOM_STATE)

display(len(df))
display(len(train_texts))
display(len(test_texts))
display(len(val_texts))

1601127

1280901

160113

160113

In [5]:
# загружаем токенизатор
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# тренировочный, тестовый и валидационный датасеты
train_dataset = NextTokenDataset(train_texts, tokenizer, seq_len=SEQ_LEN)
test_dataset = NextTokenDataset(test_texts, tokenizer, seq_len=SEQ_LEN)
val_dataset = NextTokenDataset(val_texts, tokenizer, seq_len=SEQ_LEN)

# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256) 
test_loader = DataLoader(test_dataset, batch_size=256) 

vocab_size = tokenizer.vocab_size

Создадим и обучим первую модель. Ее параметры: 
hidden_dim=128, batch=256, lr=1e-3, без dropout 

In [6]:
# создаем и обучаем модель
model1 = LSTMAutocomplete(vocab_size, hidden_dim=128, eos_token_id=tokenizer.sep_token_id)
trainer = LSTMTrainer(model1, tokenizer, vocab_size)

trainer.fit(train_loader, val_loader, n_epochs=NUM_EPOCHS)

Device: cuda
------------------------------


100%|██████████| 8824/8824 [04:02<00:00, 36.45it/s]


Epoch 1 | Train Loss: 5.487 | Val Loss: 5.266
------------------------------


100%|██████████| 1118/1118 [11:18<00:00,  1.65it/s]


Metrics
rouge1: 0.0640
rouge2: 0.0061
rougeL: 0.0628
rougeLsum: 0.0628


...

Metrics
rouge1: 0.0624
rouge2: 0.0061
rougeL: 0.0611
rougeLsum: 0.0611


100%|██████████| 8824/8824 [04:00<00:00, 36.71it/s]


Epoch 3 | Train Loss: 4.891 | Val Loss: 5.163
------------------------------


100%|██████████| 1118/1118 [11:00<00:00,  1.69it/s]


Metrics
rouge1: 0.0662
rouge2: 0.0067
rougeL: 0.0648
rougeLsum: 0.0648


Рассмотрим сгенерированные токены:

In [12]:
# сохраним веса
PATH = 'models/128_1e-3_nDO.pth'
torch.save(model1.state_dict(), PATH)

In [8]:
# сгенерируем несколько токенов
prefix = torch.tensor(tokenizer.encode("i want to", add_special_tokens=False))
generated_ids = model1.generate(prefix, max_new_tokens=2)
print(tokenizer.decode(generated_ids.tolist()))

prefix = torch.tensor(tokenizer.encode("my body feels", add_special_tokens=False))
generated_ids = model1.generate(prefix, max_new_tokens=2)
print(tokenizer.decode(generated_ids.tolist()))

prefix = torch.tensor(tokenizer.encode("you shoulda got", add_special_tokens=False))
generated_ids = model1.generate(prefix, max_new_tokens=2)
print(tokenizer.decode(generated_ids.tolist()))

i want to go to
my body feels like i
you shoulda got a new


Заметим, что метрика ROUGE указывает на далеко не самые лучшие результаты, но генерация выглядит довольно осмысленно.

Первая модель обучена. Теперь проведем подбор параметров. Выше мы обучали модель со следующими параметрами:
1. hidden_dim=128, batch=256, lr=1e-3, без dropout 
Видно, как отскакивает loss.

Сравним со следующими параметрами:

2. hidden_dim=128, batch=256, lr=5e-4, dropout=0.3 - сравним, будет ли обучаться стабильнее
3. hidden_dim=256, batch=256, lr=1e-3, без dropout - посмотрим, будут ли улучшения в связи с увеличением сложности модели
4. hidden_dim=256, batch=256, lr=5e-4, dropout=0.3 - добавим дропаут, понаблюдаем за стабильностью обучения

In [9]:
params = [
    {"name": "128_5e-4_DO", "hidden_dim": 128, "lr": 5e-4, "dropout": 0.3, "num_layers": 2},
    {"name": "256_1e-3_nDO",  "hidden_dim": 256, "lr": 1e-3, "dropout": 0.0, "num_layers": 1},
    {"name": "256_5e-4_DO", "hidden_dim": 256, "lr": 5e-4, "dropout": 0.3, "num_layers": 2},
]

for par in params:
    print("=" * 30)
    print(par["name"])
    print("=" * 30)

    model = LSTMAutocomplete(vocab_size, hidden_dim=par["hidden_dim"], eos_token_id=tokenizer.sep_token_id, dropout=par["dropout"], num_layers=par["num_layers"])
    trainer = LSTMTrainer(model, tokenizer, vocab_size, lr=par["lr"])

    trainer.fit(train_loader, val_loader, n_epochs=NUM_EPOCHS)

    # сохраним веса
    PATH = 'models/' + par["name"] + '.pth'
    torch.save(model.state_dict(), PATH)    

    # сгенерируем несколько токенов
    prefix = torch.tensor(tokenizer.encode("i want to", add_special_tokens=False))
    generated_ids = model.generate(prefix, max_new_tokens=2)
    print(tokenizer.decode(generated_ids.tolist()))

    prefix = torch.tensor(tokenizer.encode("my body feels", add_special_tokens=False))
    generated_ids = model.generate(prefix, max_new_tokens=2)
    print(tokenizer.decode(generated_ids.tolist()))

    prefix = torch.tensor(tokenizer.encode("you shoulda got", add_special_tokens=False))
    generated_ids = model.generate(prefix, max_new_tokens=2)
    print(tokenizer.decode(generated_ids.tolist()))


128_5e-4_DO
Device: cuda
------------------------------


100%|██████████| 8824/8824 [04:08<00:00, 35.47it/s]


Epoch 1 | Train Loss: 6.263 | Val Loss: 5.636
------------------------------


100%|██████████| 1118/1118 [12:20<00:00,  1.51it/s]


Metrics
rouge1: 0.0665
rouge2: 0.0058
rougeL: 0.0654
rougeLsum: 0.0654


...

Metrics
rouge1: 0.0699
rouge2: 0.0074
rougeL: 0.0685
rougeLsum: 0.0685


100%|██████████| 8824/8824 [05:38<00:00, 26.03it/s]


Epoch 3 | Train Loss: 4.926 | Val Loss: 5.017
------------------------------


100%|██████████| 1118/1118 [17:17<00:00,  1.08it/s]


Metrics
rouge1: 0.0705
rouge2: 0.0077
rougeL: 0.0690
rougeLsum: 0.0690
i want to go to
my body feels like i
you shoulda got a new


In [10]:
# 1) Загрузка токенизатора и модели
model_name = "distilgpt2"          # лёгкая версия GPT-2
tokenizer_gpt = AutoTokenizer.from_pretrained(model_name)
model_gpt = AutoModelForCausalLM.from_pretrained(model_name)

# 2) Создаём pipeline для генерации
generator = pipeline(
    task="text-generation",
    model=model_gpt,
    tokenizer=tokenizer_gpt,
    device=0  # -1 = CPU; 0 = первый GPU (если есть)
)

# 3) Тестовые промпты
prompts = ["i want to", "my body feels", "you shoulda got"]

# 4) Генерируем продолжения
for p in prompts:
    out = generator(
        p,
        max_new_tokens=2,       # итоговая длина (включая prompt)
        num_return_sequences=1,
        do_sample=True,      # стохастическая генерация
        top_p=0.95,          # nucleus sampling
        temperature=0.8
    )

    print(out[0]["generated_text"]) 

# оцениваем "изкоробочный" трансформер
results_gpt = evaluate_gpt2_rouge(
    generator=generator,
    texts=val_texts,
    tokenizer=tokenizer_gpt,
    max_new_tokens=50
)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i want to talk about
my body feels very hot
you shoulda got a new


100%|██████████| 160113/160113 [32:48<00:00, 81.33it/s] 


Metric: GPT2 ROUGE
rouge1: 0.0579
rouge2: 0.0064
rougeL: 0.0579
rougeLsum: 0.0578


ВЫБОР ЛУЧШЕЙ МОДЕЛИ

Рассмотрим результаты обучения моделей.

Модель 1 показывает стабильное обучение, качество среднее, при ручном просмотре результат генерации пригоден для использования.

Модель 2 отличается дропаутом, но метрики практически не отличаются. Возможно, дропаут не дает результатов ввиду той же размерности.

Модель 3 с hidden_dim=256 без дропаута демонстрирует улучшение качества по метрикам. Ручной просмотр сгенерированных примеров показывает такие же, как для предыдущих двух моделей, результаты.

Модель 4 с hidden_dim=256 с дропаутом показывает еще больше улучшения по метрикам, но просмотр сгенерированных данных вручную показал те же результаты. Нужно отметить, что данная модель самая "тяжелая" и медленнее всего обучается.

Предобученная модель distilgpt2 показывает результаты хуже. Просмотр сгенерированных вручную результатов также показывает, что несмотря на пригодность distilgpt2 в целом, LSTM модели дают более натуральный с точки зрения ожиданий результат. Вероятно, это можно объяснить тем, что distilgpt2 не дообучалась на исходном датасете (данные могли различаться по стилю и т.д.). 

ВЫВОДЫ:

Так как по условиям задачи ввиду использования на мобильных устройствах необходимо выбрать самую легковесную модель, предлагается модель 1 со следующими характеристиками: hidden_dim=128, batch=256, lr=1e-3, без dropout.

distilgpt2 значительно тяжелее и плохо подошла бы для мобильных устройств.

Примеры предсказаний выбранной модели из запуска выше:
i want to go to
my body feels like i
you shoulda got a new

Данные предсказания подходят для коммуникаций в социальных сетях.

Рассмотрим метрику ROUGE для выбранной модели на тестовом датасете:

In [11]:
# ROUGE на тестовой выборке для лучшей модели
rouge_scores_best = evaluate_rouge(
                model1, test_loader, tokenizer
            )

100%|██████████| 1108/1108 [10:54<00:00,  1.69it/s]


Metrics
rouge1: 0.0655
rouge2: 0.0065
rougeL: 0.0642
rougeLsum: 0.0642


Тестовые метрики близки к валидационным: можно сделать вывод, что модель способна обобщать.

Данная модель является рабочим компромиссом между качеством и легкостью, пригодна для автодополнения на мобильных устройствах.